# Workshop Notebook 7: Deploy Pipeline to Edge Devices

For this workshop, we will take a Wallaroo pipeline and publish it to an Open Container (OCI) Registry.  The registry details are stored in the Wallaroo instance as the Edge Registry.  

In this set of exercises, you will:

1. Use a pre-trained model and deploy it to Wallaroo.
1. Perform sample inferences.
1. Publish the pipeline to the Edge Registry.
1. See the steps to deploy the published pipeline to an Edge device and perform inferences through it.

Deployment to the Edge allows data scientists to work in Wallaroo to test their models in Wallaroo, then once satisfied with the results publish those pipelines.  DevOps engineers then take those published pipeline details from the Edge registry and deploy them into Docker and Kubernetes environments.

This workshop will demonstrate the following concepts:

* [Wallaroo Workspaces](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-workspace/):  Workspaces are environments were users upload models, create pipelines and other artifacts.  The workspace should be considered the fundamental area where work is done.  Workspaces are shared with other users to give them access to the same models, pipelines, etc.
* [Wallaroo Model Upload and Registration](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-model-uploads/): ML Models are uploaded to Wallaroo through the SDK or the MLOps API to a **workspace**.  ML models include default runtimes (ONNX, Python Step, and TensorFlow) that are run directly through the Wallaroo engine, and containerized runtimes (Hugging Face, PyTorch, etc) that are run through in a container through the Wallaroo engine.
* [Wallaroo Pipelines](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/): Pipelines are used to deploy models for inferencing.  Each model is a **pipeline step** in a pipelines, where the inputs of the previous step are fed into the next.  Pipeline steps can be ML models, Python scripts, or Arbitrary Python (these contain necessary models and artifacts for running a model).
* [Pipeline Edge Publication](https://docs.wallaroo.ai/20230300/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline-publication/): How to publish a Wallaroo pipeline to an OCI registry, then deploy that pipeline into other environments.

For this tutorial, we will be providing pre-trained models in ONNX format, and have connected a sample Edge Registry to our Wallaroo instance.

For more Wallaroo procedures, see the [Wallaroo Documentation site](https://docs.wallaroo.ai).

## Preliminaries

In the blocks below we will preload some required libraries.

For convenience, the following `helper functions` are defined to retrieve previously created workspaces, models, and pipelines:

* `get_workspace(name, client)`: This takes in the name and the Wallaroo client being used in this session, and returns the workspace matching `name`.  If no workspaces are found matching the name, raises a `KeyError` and returns `None`.
* `get_model_version(model_name, workspace)`: Retrieves the most recent model version from the model matching the `model_name` within the provided `workspace`.  If no model matches that name, raises a `KeyError` and returns `None`.
* `get_pipeline(pipeline_name, workspace)`: Retrieves the most pipeline from the workspace matching the `pipeline_name` within the provided `workspace`.  If no model matches that name, raises a `KeyError` and returns `None`.

In [1]:
import json
import os
import datetime

import wallaroo
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework

# used to display dataframe information without truncating
from IPython.display import display
import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth', None)

import time
import pyarrow as pa

### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [2]:
## blank space to log in 

wl = wallaroo.Client()

### Set Configurations

Set the workspace, pipeline, and model used from Notebook 1.  The helper functions will make this task easier.

#### Set Configurations References

* [Wallaroo SDK Essentials Guide: Workspace Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-workspace/)
* [Wallaroo SDK Essentials Guide: Pipeline Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline/)

In [3]:
# retrieve the previous workspace, model, and pipeline version

workspace_name = "workshop-workspace-john-05"

workspace = wl.get_workspace(name=workspace_name)

# set your current workspace to the workspace that you just created
wl.set_current_workspace(workspace)

# optionally, examine your current workspace
wl.get_current_workspace()

model_name = 'house-price-prime'

prime_model_version = wl.get_model(model_name)

pipeline_name = 'houseprice-estimator'

pipeline = wl.get_pipeline(pipeline_name)

display(workspace)
display(prime_model_version)
display(pipeline)


{'name': 'workshop-workspace-john-05', 'id': 11, 'archived': False, 'created_by': '76b893ff-5c30-4f01-bd9e-9579a20fc4ea', 'created_at': '2024-04-22T20:33:29.332164+00:00', 'models': [{'name': 'house-price-prime', 'versions': 2, 'owner_id': '""', 'last_update_time': datetime.datetime(2024, 4, 22, 20, 47, 17, 816549, tzinfo=tzutc()), 'created_at': datetime.datetime(2024, 4, 22, 20, 34, 37, 434083, tzinfo=tzutc())}, {'name': 'house-price-rf-model', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2024, 4, 22, 20, 53, 36, 195788, tzinfo=tzutc()), 'created_at': datetime.datetime(2024, 4, 22, 20, 53, 36, 195788, tzinfo=tzutc())}, {'name': 'house-price-gbr-model', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2024, 4, 22, 20, 53, 39, 543410, tzinfo=tzutc()), 'created_at': datetime.datetime(2024, 4, 22, 20, 53, 39, 543410, tzinfo=tzutc())}], 'pipelines': [{'name': 'houseprice-estimator', 'create_time': datetime.datetime(2024, 4, 22, 20, 34, 49, 55

Name,house-price-prime
Version,be93bbfc-4f0c-444d-aa89-5f15dec01d83
File Name,xgb_model.onnx
SHA,31e92d6ccb27b041a324a7ac22cf95d9d6cc3aa7e8263a229f7c4aec4938657c
Status,ready
Image Path,None
Architecture,x86
Acceleration,none
Updated At,2024-22-Apr 20:47:17


name,houseprice-estimator
created,2024-04-22 20:34:49.559433+00:00
last_updated,2024-04-24 16:18:40.930021+00:00
deployed,False
arch,x86
accel,none
tags,
versions,"eb94fe18-363a-498d-bd36-943289f0f74d, 4d4fdba1-dc5d-4c08-9375-e08ce8738730, b68ae767-dd6b-4f09-a196-a96d2cdaefb1, 3eedd163-bd54-4bde-943e-3f8b32a1f47c, 957ea8c5-d5c0-4629-82fb-fde09cf06958, e6fc1ebc-1a8f-48d2-9709-41c6820d6158, d0517194-4275-4684-860b-b35d64efba0a, 95cf17b2-19b1-40b6-82a5-d50b7f78094d, fad60d26-5f2f-467b-82ab-b97e60cfae2a, 71d8ae65-6b5e-422e-9d91-90fed507f74a, d80ef023-a703-4822-910d-b84f8d20174d, b99c6d8c-1d57-49ce-9690-07fa8f6988db, fbdf00f2-104e-41c6-9df7-86127bd2322e, 1ba316a5-6523-47f1-9e44-ec4c30e5710a"
steps,house-price-prime
published,False


## Deploy the Pipeline with the Model Version Step

As per the other workshops:

1. Clear the pipeline of all steps.
1. Add the model version as a pipeline step.
1. Deploy the pipeline with the following deployment configuration:

```python
deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(0.5).memory("1Gi").build()
```

In [4]:
deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(0.5).memory("1Gi").build()
pipeline.deploy(deployment_config=deploy_config)

name,houseprice-estimator
created,2024-04-22 20:34:49.559433+00:00
last_updated,2024-04-24 16:26:01.820203+00:00
deployed,True
arch,x86
accel,none
tags,
versions,"6912f5f1-6fbe-4795-8cec-033d97291a18, eb94fe18-363a-498d-bd36-943289f0f74d, 4d4fdba1-dc5d-4c08-9375-e08ce8738730, b68ae767-dd6b-4f09-a196-a96d2cdaefb1, 3eedd163-bd54-4bde-943e-3f8b32a1f47c, 957ea8c5-d5c0-4629-82fb-fde09cf06958, e6fc1ebc-1a8f-48d2-9709-41c6820d6158, d0517194-4275-4684-860b-b35d64efba0a, 95cf17b2-19b1-40b6-82a5-d50b7f78094d, fad60d26-5f2f-467b-82ab-b97e60cfae2a, 71d8ae65-6b5e-422e-9d91-90fed507f74a, d80ef023-a703-4822-910d-b84f8d20174d, b99c6d8c-1d57-49ce-9690-07fa8f6988db, fbdf00f2-104e-41c6-9df7-86127bd2322e, 1ba316a5-6523-47f1-9e44-ec4c30e5710a"
steps,house-price-prime
published,False


### Sample Inference

Verify the pipeline is deployed properly with a sample inference with the file `./data/test_data.df.json`.

In [5]:
# sample inference from previous code here

pipeline.infer_from_file('../data/test_data.df.json')

,time,in.tensor,out.variable,anomaly.count
0,2024-04-24 16:26:24.873,"[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]",[659806.0],0
1,2024-04-24 16:26:24.873,"[2.0, 2.5, 2170.0, 6361.0, 1.0, 0.0, 2.0, 3.0, 8.0, 2170.0, 0.0, 47.7109, -122.017, 2310.0, 7419.0, 6.0, 0.0, 0.0]",[732883.5],0
2,2024-04-24 16:26:24.873,"[3.0, 2.5, 1300.0, 812.0, 2.0, 0.0, 0.0, 3.0, 8.0, 880.0, 420.0, 47.5893, -122.317, 1300.0, 824.0, 6.0, 0.0, 0.0]",[419508.84],0
3,2024-04-24 16:26:24.873,"[4.0, 2.5, 2500.0, 8540.0, 2.0, 0.0, 0.0, 3.0, 9.0, 2500.0, 0.0, 47.5759, -121.994, 2560.0, 8475.0, 24.0, 0.0, 0.0]",[634028.75],0
4,2024-04-24 16:26:24.873,"[3.0, 1.75, 2200.0, 11520.0, 1.0, 0.0, 0.0, 4.0, 7.0, 2200.0, 0.0, 47.7659, -122.341, 1690.0, 8038.0, 62.0, 0.0, 0.0]",[427209.47],0
...,...,...,...,...
3995,2024-04-24 16:26:24.873,"[4.0, 2.25, 2620.0, 98881.0, 1.0, 0.0, 0.0, 3.0, 7.0, 1820.0, 800.0, 47.4662, -122.453, 1728.0, 95832.0, 63.0, 0.0, 0.0]",[436151.13],0
3996,2024-04-24 16:26:24.873,"[3.0, 2.5, 2244.0, 4079.0, 2.0, 0.0, 0.0, 3.0, 7.0, 2244.0, 0.0, 47.2606, -122.254, 2077.0, 4078.0, 3.0, 0.0, 0.0]",[284810.7],0
3997,2024-04-24 16:26:24.873,"[3.0, 1.75, 1490.0, 5000.0, 1.0, 0.0, 1.0, 3.0, 8.0, 1250.0, 240.0, 47.5257, -122.392, 1980.0, 5000.0, 61.0, 0.0, 0.0]",[575571.44],0
3998,2024-04-24 16:26:24.873,"[4.0, 2.5, 2740.0, 5700.0, 2.0, 0.0, 0.0, 3.0, 9.0, 2740.0, 0.0, 47.3535, -122.026, 3010.0, 5281.0, 8.0, 0.0, 0.0]",[432262.38],0


## Undeploying Your Pipeline

You should always undeploy your pipelines when you are done with them, or don't need them for a while. This releases the resources that the pipeline is using for other processes to use. You can always redeploy the pipeline when you need it again. As a reminder, here are the commands to deploy and undeploy a pipeline:

```python

# "turn off" the pipeline and releaase its resources
my_pipeline.undeploy()
```

In [7]:
# blank space to undeploy the pipeline
pipeline.undeploy()

name,houseprice-estimator
created,2024-04-22 20:34:49.559433+00:00
last_updated,2024-04-24 16:26:01.820203+00:00
deployed,False
arch,x86
accel,none
tags,
versions,"6912f5f1-6fbe-4795-8cec-033d97291a18, eb94fe18-363a-498d-bd36-943289f0f74d, 4d4fdba1-dc5d-4c08-9375-e08ce8738730, b68ae767-dd6b-4f09-a196-a96d2cdaefb1, 3eedd163-bd54-4bde-943e-3f8b32a1f47c, 957ea8c5-d5c0-4629-82fb-fde09cf06958, e6fc1ebc-1a8f-48d2-9709-41c6820d6158, d0517194-4275-4684-860b-b35d64efba0a, 95cf17b2-19b1-40b6-82a5-d50b7f78094d, fad60d26-5f2f-467b-82ab-b97e60cfae2a, 71d8ae65-6b5e-422e-9d91-90fed507f74a, d80ef023-a703-4822-910d-b84f8d20174d, b99c6d8c-1d57-49ce-9690-07fa8f6988db, fbdf00f2-104e-41c6-9df7-86127bd2322e, 1ba316a5-6523-47f1-9e44-ec4c30e5710a"
steps,house-price-prime
published,False


## Publish the Pipeline for Edge Deployment

It worked! For a demo, we'll take working once as "tested". So now that we've tested our pipeline, we are ready to publish it for edge deployment.

Publishing it means assembling all of the configuration files and model assets and pushing them to an Open Container Initiative (OCI) repository set in the Wallaroo instance as the Edge Registry service.  DevOps engineers then retrieve that image and deploy it through Docker, Kubernetes, or similar deployments.

See [Edge Deployment Registry Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-edge-deployment/) for details on adding an OCI Registry Service to Wallaroo as the Edge Deployment Registry.

This is done through the SDK command `wallaroo.pipeline.publish(deployment_config)` which has the following parameters and returns.

#### Publish a Pipeline Parameters

The `publish` method takes the following parameters.  The containerized pipeline will be pushed to the Edge registry service with the model, pipeline configurations, and other artifacts needed to deploy the pipeline.

| Parameter | Type | Description |
|---|---|---|
| `deployment_config` | `wallaroo.deployment_config.DeploymentConfig` (*Optional*) | Sets the pipeline deployment configuration.  For example:    For more information on pipeline deployment configuration, see the [Wallaroo SDK Essentials Guide: Pipeline Deployment Configuration](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline-deployment-config/).

#### Publish a Pipeline Returns

| Field | Type | Description |
|---|---|---|
| id | integer | Numerical Wallaroo id of the published pipeline. |
| pipeline version id | integer | Numerical Wallaroo id of the pipeline version published. |
| status | string | The status of the pipeline publication.  Values include:  <ul><li>PendingPublish: The pipeline publication is about to be uploaded or is in the process of being uploaded.</li><li>Published:  The pipeline is published and ready for use.</li></ul> |
| Engine URL | string | The URL of the published pipeline engine in the edge registry. |
| Pipeline URL | string | The URL of the published pipeline in the edge registry. |
| Helm Chart URL | string | The URL of the helm chart for the published pipeline in the edge registry. |
| Helm Chart Reference | string | The help chart reference. |
| Helm Chart Version | string | The version of the Helm Chart of the published pipeline.  This is also used as the Docker tag. |
| Engine Config | `wallaroo.deployment_config.DeploymentConfig` | The pipeline configuration included with the published pipeline. |
| Created At | DateTime | When the published pipeline was created. |
| Updated At | DateTime | When the published pipeline was updated. |

### Publish the Pipeline for Edge Deployment Exercise

We will now publish the pipeline to our Edge Deployment Registry with the `pipeline.publish(deployment_config)` command.  `deployment_config` is an optional field that specifies the pipeline deployment.  This can be overridden by the DevOps engineer during deployment.

In this example, assuming that the pipeline was saved to the variable `my_pipeline`, we would publish it to the Edge Registry already stored in the Wallaroo instance and store the pipeline publish to the variable `my_pub` with the following command:

```python
my_pub=pipeline.publish(deploy_config)
# display the publish
my_pub
```


In [8]:
## blank space to publish the pipeline

my_pub=pipeline.publish(deploy_config)
# display the publish
my_pub


Waiting for pipeline publish... It may take up to 600 sec.
Pipeline is publishing...... Published.


ID,18
Pipeline Name,houseprice-estimator
Pipeline Version,42cb257d-3d21-4b71-9645-74919b4c862f
Status,Published
Engine URL,ghcr.io/wallaroolabs/doc-samples/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2024.1.0-main-4963
Pipeline URL,ghcr.io/wallaroolabs/doc-samples/pipelines/houseprice-estimator:42cb257d-3d21-4b71-9645-74919b4c862f
Helm Chart URL,oci://ghcr.io/wallaroolabs/doc-samples/charts/houseprice-estimator
Helm Chart Reference,ghcr.io/wallaroolabs/doc-samples/charts@sha256:19dd28fb102d068af5f3bfaff3c3b149ee18ff900bb064d50447fa2a1e78996a
Helm Chart Version,0.0.1-42cb257d-3d21-4b71-9645-74919b4c862f
Engine Config,"{'engine': {'resources': {'limits': {'cpu': 1.0, 'memory': '512Mi'}, 'requests': {'cpu': 1.0, 'memory': '512Mi'}, 'accel': 'none', 'arch': 'x86', 'gpu': False}}, 'engineAux': {'autoscale': {'type': 'none'}, 'images': {}}}"
User Images,[]


## List Published Pipelines

The method `wallaroo.client.list_pipelines()` shows a list of all pipelines in the Wallaroo instance, and includes the `published` field that indicates whether the pipeline was published to the registry (`True`), or has not yet been published (`False`).

### List Published Pipelines Exercise

List all pipelines and see which ones are published or not.  For example, if your client was saved to the variable `wl`, then the following will list the pipelines and display which ones are published.

```python
wl.list_pipelines()
```

In [9]:
# list the pipelines and view which are published

wl.list_pipelines()

[{'name': 'edge-low-connection-demonstration', 'create_time': datetime.datetime(2024, 4, 23, 17, 1, 22, 48776, tzinfo=tzutc()), 'definition': '[]'},
 {'name': 'edge-low-connection-demonstration', 'create_time': datetime.datetime(2024, 4, 23, 16, 10, 34, 637751, tzinfo=tzutc()), 'definition': '[]'},
 {'name': 'houseprice-estimator', 'create_time': datetime.datetime(2024, 4, 22, 20, 34, 49, 559433, tzinfo=tzutc()), 'definition': '[]'},
 {'name': 'edge-pipeline', 'create_time': datetime.datetime(2024, 4, 22, 18, 26, 6, 294570, tzinfo=tzutc()), 'definition': '[]'},
 {'name': 'edge-pipeline', 'create_time': datetime.datetime(2024, 4, 22, 18, 8, 21, 762528, tzinfo=tzutc()), 'definition': '[]'},
 {'name': 'edge-low-connection-demo', 'create_time': datetime.datetime(2024, 4, 22, 17, 53, 56, 176500, tzinfo=tzutc()), 'definition': '[]'},
 {'name': 'edge-pipeline-classification-cybersecurity', 'create_time': datetime.datetime(2024, 4, 22, 17, 4, 23, 58677, tzinfo=tzutc()), 'definition': '[]'},
 {'name': 'vgg16-clustering-pipeline', 'create_time': datetime.datetime(2024, 4, 22, 16, 17, 8, 651909, tzinfo=tzutc()), 'definition': '[]'},
 {'name': 'new-edge-inline-replacement', 'create_time': datetime.datetime(2024, 4, 22, 15, 43, 4, 574641, tzinfo=tzutc()), 'definition': '[]'},
 {'name': 'edge-inline-replacement-demon', 'create_time': datetime.datetime(2024, 4, 22, 15, 27, 36, 828826, tzinfo=tzutc()), 'definition': '[]'}]

## List Publishes from a Pipeline

All publishes created from a pipeline are displayed with the `wallaroo.pipeline.publishes` method.  The `pipeline_version_id` is used to know what version of the pipeline was used in that specific publish.  This allows for pipelines to be updated over time, and newer versions to be sent and tracked to the Edge Deployment Registry service.

### List Publishes Parameters

N/A

### List Publishes Returns

A List of the following fields:

| Field | Type | Description |
|---|---|---|
| id | integer | Numerical Wallaroo id of the published pipeline. |
| pipeline_version_id | integer | Numerical Wallaroo id of the pipeline version published. |
| engine_url | string | The URL of the published pipeline engine in the edge registry. |
| pipeline_url | string | The URL of the published pipeline in the edge registry. |
| created_by | string | The email address of the user that published the pipeline.
| Created At | DateTime | When the published pipeline was created. |
| Updated At | DateTime | When the published pipeline was updated. |

### List Publishes from a Pipeline Exercise

List all of the publishes from our pipeline.  For example, if our pipeline is `my_pipeline`, then we would list all publishes from the pipeline with the following:

```python
my_pipeline.publishes()
```

In [10]:
pipeline.publishes()

id,pipeline_version_name,engine_url,pipeline_url,created_by,created_at,updated_at
18,42cb257d-3d21-4b71-9645-74919b4c862f,ghcr.io/wallaroolabs/doc-samples/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2024.1.0-main-4963,ghcr.io/wallaroolabs/doc-samples/pipelines/houseprice-estimator:42cb257d-3d21-4b71-9645-74919b4c862f,john.hummel@wallaroo.ai,2024-24-Apr 16:27:58,2024-24-Apr 16:27:58


## Congratulations!

You have now 

* Created a workspace and set it as the current workspace.
* Uploaded an ONNX model.
* Created a Wallaroo pipeline, and set the most recent version of the uploaded model as a pipeline step.
* Successfully send data to your pipeline for inference through the SDK and through an API call.


## DevOps - Pipeline Edge Deployment

Once a pipeline is deployed to the Edge Registry service, it can be deployed in environments such as Docker, Kubernetes, or similar container running services by a DevOps engineer.  Docker Run and Helm Install templates are provided as part of the pipeline publish display from the Wallaroo SDK.

## Edge Deployed Pipeline API Endpoints

Once deployed, we can check the pipelines and models available.  We'll use a `curl` command, but any HTTP based request will work the same way.

### Pipelines Endpoints

The endpoint `/pipelines` returns:

* **id** (*String*):  The name of the pipeline.
* **status** (*String*):  The status as either `Running`, or `Error` if there are any issues.

For this example, the deployment is made on a machine called `testboy.local`.  Replace this URL with the URL of you edge deployment.

#### Pipelines Endpoints Exercise

Use the following `curl` command to view the pipeline data.  For example, if the pipeline was deployed on `localhost`, then the command would be:

```bash
!curl locahost:8080/pipelines
```

In [14]:
# blank space to run the command - replace testboy.local with the host

!curl testboy.local:8080/pipelines

{"pipelines":[{"id":"houseprice-estimator","status":"Running"}]}

### Models Endpoints

The endpoint `/models` returns a List of models with the following fields:

* **name** (*String*): The model name.
* **sha** (*String*): The sha hash value of the ML model.
* **status** (*String*):  The status of either Running or Error if there are any issues.
* **version** (*String*):  The model version.  This matches the version designation used by Wallaroo to track model versions in UUID format.

#### Models Endpoints Exercise

Use the following `curl` command to view the models data.  For example, if the pipeline was deployed on `localhost`, then the command would be:

```bash
!curl locahost:8080/models
```

In [15]:
# blank space to run the command - replace testboy.local with the host

!curl testboy.local:8080/models

{"models":[{"name":"house-price-prime","sha":"31e92d6ccb27b041a324a7ac22cf95d9d6cc3aa7e8263a229f7c4aec4938657c","status":"Running","version":"e49c245b-be7b-48a6-a6b8-93ac4c1fe69a"}]}

### Edge Deployed Inference

The inference endpoint takes the following pattern:

* `/pipelines/{pipeline-name}`:  The `pipeline-name` is the same as returned from the [`/pipelines`](#list-pipelines) endpoint as `id`.

Wallaroo inference endpoint URLs accept the following data inputs through the `Content-Type` header:

* `Content-Type: application/vnd.apache.arrow.file`: For Apache Arrow tables.
* `Content-Type: application/json; format=pandas-records`: For pandas DataFrame in record format.

It returns a `application/json; format=pandas-records` - the same pandas record we've been working with.

### Edge Deployed Inference Exercise

Perform an inference on the deployed pipeline using `curl`.  This command will look like this:

```bash
!curl -X POST localhost:8080/pipelines/{YOUR PIPELINE NAME} -H "Content-Type: application/json; format=pandas-records" --data @../data/singleton.df.json
```

In [16]:
!curl -X POST testboy.local:8080/pipelines/houseprice-estimator -H "Content-Type: application/json; format=pandas-records" --data @../data/singleton.df.json

[{"check_failures":[],"elapsed":[120996489,42784927],"model_name":"house-price-prime","model_version":"e49c245b-be7b-48a6-a6b8-93ac4c1fe69a","original_data":null,"outputs":[{"Float":{"data":[2176827.0],"dim":[1,1],"v":1}}],"pipeline_name":"houseprice-estimator","shadow_data":{},"time":1694628519169}]